<a href="https://colab.research.google.com/github/rhmes/point-cloud-compression/blob/main/point_cloud_compression_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Point Cloud Compression Demo

This notebook demonstrates installation, core functionality, and testing for the point cloud compression project.

In [ ]:
# Connect to Google Drive (for Colab users)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Google Drive mounted.')
except ImportError:
    print('Not running in Colab, skipping Google Drive mount.')

# Check if venv_cpu or venv_gpu exists, run setup if not
import os
if not (os.path.exists('venv_cpu') or os.path.exists('venv_gpu')):
    print('No virtual environment found. Running setup...')
    os.system('bash venv_gpu_setup.sh')  # Change to venv_cpu_setup.sh if needed
else:
    print('Virtual environment already exists. Skipping setup.')

In [ ]:
# Check if datasets exist and download if missing
import os
import urllib.request

def check_and_download_dataset(dataset_dir, url=None):
    if os.path.exists(dataset_dir) and os.listdir(dataset_dir):
        print(f"Dataset found in '{dataset_dir}'.")
        return True
    if url:
        print(f"Downloading dataset to '{dataset_dir}'...")
        os.makedirs(dataset_dir, exist_ok=True)
        # Example: download and extract logic (customize for your dataset)
        zip_path = os.path.join(dataset_dir, 'dataset.zip')
        urllib.request.urlretrieve(url, zip_path)
        import zipfile
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(dataset_dir)
        os.remove(zip_path)
        print("Download and extraction complete.")
        return True
    print(f"Dataset not found and no URL provided for '{dataset_dir}'.")
    return False

# Example usage (customize URLs as needed):
check_and_download_dataset('data/ModelNet40', url=None)  # Add URL if available

In [ ]:
# Example core functionality: List files in the data directory

def list_data_files(data_dir='data'):
    if not os.path.exists(data_dir):
        print(f"Directory '{data_dir}' does not exist.")
        return []
    files = os.listdir(data_dir)
    print(f"Files in '{data_dir}':", files)
    return files

# Run the function
data_files = list_data_files()

In [ ]:
# Compress point cloud test files using the trained model
!python compress.py './data/ModelNet40_pc_01_8192p/**/test/*.ply' './data/ModelNet40_K256_compressed' './model/K256' --K 256

In [ ]:
# Decompress the compressed point cloud files
!python decompress.py './data/ModelNet40_K256_compressed' './data/ModelNet40_K256_decompressed' './model/K256' --K 256

In [ ]:
# Evaluate the compression results using PSNR, Chamfer distance, and bpp metrics
!python eval.py './data/ModelNet40_pc_01_8192p/**/test/*.ply' './data/ModelNet40_K256_compressed' './data/ModelNet40_K256_decompressed' './eval/ModelNet40_K256.csv' '../geo_dist/build/pc_error'

In [ ]:
# Visualize evaluation metrics and save plots
!python visualize.py --csv './eval/ModelNet40_K256.csv' --outdir './figure/'